|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 2:</h2>|<h1>Large language models<h1>|
|<h2>Section:</h2>|<h1>Pretrain LLMs<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Train model 5 with modifications<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests

import torch
import torch.nn as nn
import torch.nn.functional as F

# vector plots
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
# hyperparameters for GPT2-124M
n_vocab    = 50257     # GPT-2 vocab size
embed_dim  =   768     # embedding dimension
seq_len    =   256     # max sequence length
n_heads    =    12     # attention heads
n_blocks   =    12     # transformer blocks
batch_size =    16

# use GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Exercise 1: Import, tokenize, and split the text

In [ ]:
# tokenize the text
# Gulliver's travels :)
text = requests.get('https://www.gutenberg.org/cache/epub/829/pg829.txt').text
gtTokens =

In [ ]:
# train/test split

train_ratio = .9

# index to split data
test_split_point = int(train_ratio*len(gtTokens))

train_data = gtTokens # just the training tokens
test_data  = gtTokens # just the test tokens


# a function that returns a batch of data samples
def get_data_batch(training=True):

  # pick the dataset to use
  if training:
    data =
  else:
    data =

  # pick random indices to start
  ix = torch.randint(len(data)-seq_len,size=(batch_size,))

  # get the data and targets (via broadcasting outer product)
  X = data[ix[:,None] + torch.arange(seq_len)]
  y = data[ix[:,None] +
  return X,y


# example
X,y = get_data_batch()
print(f'Input data (size {X.shape}):\n',X)
print(f'\n\nTargets (size {y.shape}):\n',y)

# Exercise 2: Copy and test the model

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self):
    super().__init__()

    # number of attention heads
    self.num_heads = n_heads
    self.head_dim  = embed_dim //

    # the three Q,K,V weights matrices are initialized as one, and are split inside forward()
    self.QKV = nn.Linear(,, bias=True)

    # linear mixing after attention
    self.W0 = nn.Linear(, , bias=True)


  def forward(self,x):

    # sizes for later use
    B, T, E = x.shape # [batch, seq_len, embed_dim]

    # push data through Q, K, and V in one concatenated matrix
    qkv = self.QKV(x) # [batch, sequence, 3*embed]
    q,k,v = torch.split # each matrix is [B, T, E]

    # reshape to [B, T, nHeads, head_dim]
    #  and then transpose to [B, nHeads, T, head_dim]
    q = q.
    k = k.
    v = v.

    # Pytorch's dot-product attention function handles multi-head shapes
    out = F.scaled_dot_product_attention

    # recombine heads: (B, nHeads, T, head_dim) -> [B, T, E]
    out =

    # finally, linearly mix the attention heads using the W0 matrix
    out =

    return out




class TransformerBlock(nn.Module):
  def __init__(self):
    super().__init__()

    ### attention subblock
    self.layernorm_1 = nn.LayerNorm(embed_dim, eps=1e-5)
    self.attn = MultiHeadAttention()


    ### linear feedforward (MLP) subblock
    self.layernorm_2 = nn.LayerNorm(, eps=1e-5)
    # 4x expansion, then back to embedding size
    self.mlp_1 = nn.Linear(, , bias=True)
    self.gelu  =
    self.mlp_2 = nn.Linear(, , bias=True)

  def forward(self, x):

    # attention
    x_att =  # pre-attention normalization
    x_att =  # run through attention, then add pre-attention activation ("residual")


    # MLP
    x_ff =  # pre-MLP normalization
    x_ff =  # adjustment from expansion-contraction

    return x_ff

In [ ]:
class LanguageModel(nn.Module):
  def __init__(self):
    super().__init__()

    # token + position embeddings
    self.wte = nn.Embedding(, embed_dim) # token embedding
    self.wpe = nn.Embedding(, embed_dim) # position embedding

    # transformer blocks
    self.transformerBlocks = nn.Sequential(*[TransformerBlock() for _ in range(n_blocks)])

    # final layernorm
    self.layernorm_final =

    # lm head, with weights tied to token embedding
    self.final_head = nn.Linear(embed_dim, n_vocab, bias=False)
    self.final_head.weight = nn.Parameter()


  def forward(self, idx):

    # token + position embeddings (note the device!)
    token_emb =  # [B, T, E]
    posit_emb = self.wpe(torch.arange(idx.shape[-1],device=device)) # [T, E]
    x =  # [B, T, E]

    # pass through each transformer block
    x = self.transfo

    # final layernorm and unembeddings
    x =
    logits =  # [B, T, n_vocab]

    # scale and logsoftmax
    outputs = F.log_softmax(   /np.sqrt(embed_dim),dim=-1)

    return outputs


  def generate(self, idx, max_new_tokens=50):

    for _ in range(max_new_tokens):

      # forward pass
      logits = self()  # [B, T, n_vocab]
      logits =   # last token's logits: [B, n_vocab]

      # undo the log-softmax to get "normal" softmax (probability values)
      probs =  # [B, n_vocab]

      # sample next token
      idx_next = torch.multinomial() # [B, 1]

      # append
      idx = torch.cat(, dim=1) # [B, T+1]
    return idx


In [ ]:
# create an instance and test with some data
model = LanguageModel().to(device)

X,y = get_data_batch()
# send to GPU
out = model(X) # ~45s on cpu, <1s on gpu :D
print(f'Input size:  {}')
print(f'Output size: {}')

In [ ]:
# generate new text before any training
startToks = torch.tensor(tokenizer.encode('The man-mountain shall not depart from our dominions')).unsqueeze(0)

# text generation
Y = model.generate(,max_new_tokens=one hundred)
print(tokenizer.decode(Y[0].tolist()).replace('\r','\n'))

# Exercise 3: Train the model

In [ ]:
# create the loss and optimizer functions
loss_function = nn.NLLLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=.001, weight_decay=.01)

In [ ]:
# check loss function with sizes
X,y = get_data_batch()

log_probs =

print(f'Model input is size:   {X.shape}')
print(f'Model output is size:  {log_probs.shape}')
print(f'Target tokens is size: {y.shape}')

# flatten as before
log_probs_flat =

# compute the loss
loss = loss_function
print('\nLoss:',loss)

# Now train the model!

In [ ]:
num_samples = 501

# initialize losses
train_loss = np.zeros(num_samples)
test_loss = np.zeros(num_samples)

for sampli in range(num_samples):

  # get a batch of data


  # move data to GPU


  # clear previous gradients


  # forward pass


  # calculate the losses on the (reshaped) final target word


  # backprop


  # store the per-sample loss



  # evaluate the model with the test set
  if sampli%100==0:
    with torch.no_grad():


      # update our progress :)


In [ ]:
# plot the losses


In [ ]:
# generate new text
startToks = torch.tensor(tokenizer.encode('The man-mountain shall not depart from our dominions')).unsqueeze(0)

# text generation
Y = model.generate(startToks.to(device),max_new_tokens=100)
print(tokenizer.decode(Y[0].tolist()).replace('\r','\n'))